# Trabalho 1 - Implementação do S-DES

## Contextualização:

Este relatório é uma das atividades pedagócicas da disciplina de Segurança Computacional. Nesta matéria somos introduzidos a importantes conceitos e aplicações do ramo de segurança na computação.

Nesse sentido, este trabalho aborda um importante algoritmos de criptografia e alguns modos de operação.

O S-DES será o algoritmo a ser trabalhado e veremos seu funcionamento com dois modos de operação:
- Electronic Codebook (ECB)
- Cipher Block Chaining (CBC)

A criptografia S-DES é uma criptografia simétrica e de cifragem em blocos. É um algoritmo simplificado do DES para fins educacionais. Auxiliando, portanto, alunos a compreenderem o funcionamento do algoritmo DES através de chaves menores, funções e etapas menos complexas.

## Implementação:

Este notebook visa descrever brevemente a implementação enquanto permite a fácil execução de cada etapa. É importante ressaltar que o pdf G-SDES presente neste diretório foi usado para auxiliar na implementação e descrição das etapas necessárias.

### Visão Geral:

O algoritmo de encriptação do S-DES recebe um bloco de 8 bits e, juntamente com uma chave de 10 bits, produz um texto cifrado de 8 bits. Enquanto, o de decriptação fará o contrário, ou seja, receberá um texto de 8 bits cifrado e junto à uma chave de 10 bits retorna um texto legível de 8 bits.

Veremos a seguir que este algoritmo irá necessitar de algumas funções auxiliares para o seu funcionamento. Detalharemos elas melhor em breve, mas são elas:
1) Initial Permutation (IP)
2) fk (Função complexa)
3) SW (Switches)
4) Inverse of the Inital Permutation (IP^(-1))

Assim, a encriptação e decriptação usaram destas funções auxiliares para implementar o S-DES.

Ademais, veremos também os modos de operação ECB e CBC e iremos testá-los com algumas entradas.

## Parte 1:

Para realizar a parte 1 será necessário realizar as funções auxiliares. Fazer o funcionamento da encriptação e decriptação usando o S-DES implementado. Por fim, testaremos o algoritmos com as entradas previstas na especificação do trabalho.

Com isso, iniciaremos definindo algumas variáveis que serão utilizada em outros momentos do projeto como as S-Boxes usadas na função fk e as variáveis de teste as quais iremos inserir em cada etapa do algoritmo:

In [3]:
# S-Boxes necessary:

s_box_0 = [
    ['01', '00', '11', '10'],
    ['11', '10', '01', '00'],
    ['00', '10', '01', '11'],
    ['11', '01', '11', '10']
]

s_box_1 = [
    ['00', '01', '10', '11'],
    ['10', '00', '01', '11'],
    ['11', '00', '01', '00'],
    ['10', '01', '00', '11']
]

# Test case variables:
key = '1010000010'
block_of_data = '11010111'

### S-DES Key Generation:

Como mencionado anteriormente, o S-DES utiliza uma chave de 10 bits.

Entretanto, veremos mais a frente que para cada função fk será necessário uma subchave (SK) de 8 bits. Ou seja, a partir da chave inicial de 10 bits, será necessário uma função que gere as duas subchaves necessárias.

Para esta função Key Generation, serão necessárias outras duas funções auxiliares:
- Permutação
- Deslocamento Circular Esquerdo

#### Permutation:

A função de permutação abaixo possui dois parâmtros:
- A entrada a ser permutada
- Um vetor de permutação

Nesse sentido, a função irá, a partir do vetor de permutação rearranjar os elementos recebidos na variável de entrada. Logo, irá retornar a entrada permutada de acordo com o vetor de permutação recebido.

Ao realizar a função desta forma permitimos que ela seja usada para permutações de diversos tamanhos como veremos em breve.

In [4]:
def permutation(entry, permutation_vector):
    permutated_entry = ''.join([entry[i - 1] for i in permutation_vector])
    return permutated_entry

#### Circular Left Shift:

Esta função realiza um deslocamento circular de n bits, por isso recebe dois parâmetros:
- A entrada a ser deslocada
- A quantidade de bits para deslocar

Como especificado na S-DES, é necessário realizar este deslocamento de n bits nas duas metades da entrada. Por isso, o seguinte código foi implementado:

In [5]:
def circular_left_shift(entry, num_bits_to_shift):
    left_half_entry = entry[:5]
    right_half_entry = entry[5:]
    return left_half_entry[num_bits_to_shift:] + left_half_entry[0:num_bits_to_shift] + right_half_entry[num_bits_to_shift:] + right_half_entry[0:num_bits_to_shift]

Por fim, estas funções auxiliares definidas serão usadas com os devidos argumentos para a geração das duas subchaves que serão necessárias no S-DES.

Dessa forma, foram inseridos prints para verificar o devido funcionamento após cada etapa da função de geração das subchaves:

In [6]:
def key_generation(key):

    print(f'10-bit key: {key}')

    # First permutation of 10 bits
    p_10_permutation = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
    p_10 = permutation(key, p_10_permutation)
    print(f'After P10: {p_10}')

    # First Circular Left Shift (LS-1)
    ls_1 = circular_left_shift(p_10, 1)
    print(f'After LS-1: {ls_1}')

    # Second permuation of 8 bits
    p_8_permutation = [1, 2, 6, 3, 7, 4, 8, 5, 10, 9] # The first two bits aren't permutated
    k1 = permutation(ls_1, p_8_permutation)[2:]
    print(f'K1 = {k1}')

    # LS-2 applied to LS-1
    ls_2 = circular_left_shift(ls_1, 2)
    print(f'After LS-2: {ls_2}')

    # Third permutation of 8 bits
    k2 = permutation(ls_2, p_8_permutation)[2:]
    print(f'K2 = {k2}')

    return k1, k2

### Initial Permuation and its Inverse:

A função de permutação e sua inversa irão realizar uma permutação simples, assim a função auxiliar de permutação definida anteriormente pode ser usada para implementar estas etapas no algoritmo.

### The Function fk:

Considerada a etapa mais complexa do S-DES a função fk irá implementar diversas permutações e substituições.

Entretanto, ela pode ser resumida com a seguinte equação em que L é o lado esquerdo da entrada e R o direito:

$f_K(L, R) = \left( L \oplus F(R, SK),\ R \right)$

Para uma subchave SK qualquer, o XOR será feito bit a bit entre a saída da função "F" com o lado esquerda da entrada L.

Além disso, nota-se como o lado direito R não possui alteração na função fk.

Para devida, implementação desta etapa, faz-se necessário compreender e implementar a função F.

#### Função F:

In [7]:
def f(entry, sk):

    print(f'The input is a 4-bit number: {entry}')

    # Expansion/Permutation operation
    expansion_and_permutation_row_one = [4, 1, 2, 3]
    expansion_and_permutation_row_two = [2, 3, 4, 1]
    ep_row_one = permutation(entry, expansion_and_permutation_row_one)
    ep_row_two = permutation(entry, expansion_and_permutation_row_two)
    print(f'Matrix after E/P:\n{ep_row_one}\n{ep_row_two}')

    # SW addition using XOR
    xor_row_one = [int(n) ^ int(k) for n, k in zip(ep_row_one, sk[:4])]
    xor_row_two = [int(n) ^ int(k) for n, k in zip(ep_row_two, sk[-4:])]
    print(f'Matrix after XOR:\n{xor_row_one}\n{xor_row_two}')

    # S-boxes
    s_0_row = int(str(xor_row_one[0]) + str(xor_row_one[3]), 2)
    s_0_column = int(str(xor_row_one[1]) + str(xor_row_one[2]), 2)
    s_0_result = s_box_0[s_0_row][s_0_column]
    s_1_row = int(str(xor_row_two[0]) + str(xor_row_two[3]), 2)
    s_1_column = int(str(xor_row_two[1]) + str(xor_row_two[2]), 2)
    s_1_result = s_box_1[s_1_row][s_1_column]
    result = s_0_result + s_1_result
    print(f'After S-boxes: {result}')

    # Permutation of 4 bits
    p_4_permutation = [2, 4, 3, 1]
    p_4 = permutation(result, p_4_permutation)
    print(f'After P4: {p_4}')

    return p_4

Com isso, para finalizarmos a função fk é necessário aplicar um XOR entre o lado esquerdo e o resultado da função F. Por isso, podemos escrever a função fk como:

In [8]:
def fk(entry, sk):
    l = entry[:4]
    r = entry[-4:]
    f_result = f(r, sk)
    xor_result = ''.join([str(int(l_elem) ^ int(f_elem)) for l_elem, f_elem in zip(l, f_result)])
    print(f'Result after XOR of the F function and Left side: {xor_result}')
    return xor_result + r

### Switch Funcion:

A função SW realiza a alteração dos 4 bits da esquerda com os quatro da direita para que a segunda instância de fk possa operar nos outros 4 bits. Para isso, foi feita a seguinte função:

In [9]:
def switch(entry):
    return entry[-4:] + entry[:4]

### Encryption:

Agora, com todas funções auxiliares definidas podemos juntar tudo para fazer as funções de encriptação e decriptação do S-DES.

Como previsto, é possível notar a ordem de execução das funções auxiliares através do código da função abaixo:

In [10]:
def enc(plain_text, key):
    print(f'The message being encoded is: {plain_text}')

    # Subkey generation
    k1, k2 = key_generation(key)

    # IP
    ip_permutation = [2, 6, 3, 1, 4, 8, 5, 7]
    ip = permutation(plain_text, ip_permutation)
    print(f'After IP: {ip}')

    # First fk
    fk1 = fk(ip, k1)
    print(f'After fk1: {fk1}')

    # SW
    sw = switch(fk1)
    print(f'After SW: {sw}')

    # Second fk
    fk2 = fk(sw, k2)
    print(f'After fk2: {fk2}')

    # IP ^ (-1)
    iip_permutation = [4, 1, 3, 5, 7, 2, 8, 6]
    iip = permutation(fk2, iip_permutation)
    print(f'After the Inverse of IP: {iip}')

    print(f'The 8-bit ciphertext is: {iip}')
    return iip

enc(block_of_data, key)

The message being encoded is: 11010111
10-bit key: 1010000010
After P10: 1000001100
After LS-1: 0000111000
K1 = 10100100
After LS-2: 0010000011
K2 = 01000011
After IP: 11011101
The input is a 4-bit number: 1101
Matrix after E/P:
1110
1011
Matrix after XOR:
[0, 1, 0, 0]
[1, 1, 1, 1]
After S-boxes: 1111
After P4: 1111
Result after XOR of the F function and Left side: 0010
After fk1: 00101101
After SW: 11010010
The input is a 4-bit number: 0010
Matrix after E/P:
0001
0100
Matrix after XOR:
[0, 1, 0, 1]
[0, 1, 1, 1]
After S-boxes: 0111
After P4: 1110
Result after XOR of the F function and Left side: 0011
After fk2: 00110010
After the Inverse of IP: 10101000
The 8-bit ciphertext is: 10101000


'10101000'

Através da execução acima, é possível observar o correto funcionamento de todas funções definidas e a função principal de encriptação produz a saída correta.

### Decryption

Com isso, iremos fazer agora função de decriptação em que recebemos o texto cifrado e a chave e, com isso, conseguimos ter a mensagem legível novamente.

In [11]:
def dec(cipher_text, key):
     print(f'The message being decoded is: {cipher_text}')

     # Subkey generation
     k1, k2 = key_generation(key)

     # IP
     ip_permutation = [2, 6, 3, 1, 4, 8, 5, 7]
     ip = permutation(cipher_text, ip_permutation)
     print(f'After IP: {ip}')

     # First fk of decryption
     fk1 = fk(ip, k2)
     print(f'After fk1: {fk1}')

     # SW
     sw = switch(fk1)
     print(f'After SW: {sw}')

     # Second fk
     fk2 = fk(sw, k1)
     print(f'After fk2: {fk2}')

     # IP ^ (-1)
     iip_permutation = [4, 1, 3, 5, 7, 2, 8, 6]
     iip = permutation(fk2, iip_permutation)
     print(f'After the Inverse of IP: {iip}')

     print(f'The 8-bit plaintext is: {iip}')
     return iip

dec(enc(block_of_data, key), key)

The message being encoded is: 11010111
10-bit key: 1010000010
After P10: 1000001100
After LS-1: 0000111000
K1 = 10100100
After LS-2: 0010000011
K2 = 01000011
After IP: 11011101
The input is a 4-bit number: 1101
Matrix after E/P:
1110
1011
Matrix after XOR:
[0, 1, 0, 0]
[1, 1, 1, 1]
After S-boxes: 1111
After P4: 1111
Result after XOR of the F function and Left side: 0010
After fk1: 00101101
After SW: 11010010
The input is a 4-bit number: 0010
Matrix after E/P:
0001
0100
Matrix after XOR:
[0, 1, 0, 1]
[0, 1, 1, 1]
After S-boxes: 0111
After P4: 1110
Result after XOR of the F function and Left side: 0011
After fk2: 00110010
After the Inverse of IP: 10101000
The 8-bit ciphertext is: 10101000
The message being decoded is: 10101000
10-bit key: 1010000010
After P10: 1000001100
After LS-1: 0000111000
K1 = 10100100
After LS-2: 0010000011
K2 = 01000011
After IP: 00110010
The input is a 4-bit number: 0010
Matrix after E/P:
0001
0100
Matrix after XOR:
[0, 1, 0, 1]
[0, 1, 1, 1]
After S-boxes: 0111
A

'11010111'

Usando o texto cifrado da função de encriptação é possível notar o devido funcionamento da função de decriptação visto que o resultado retornado é o blocl de dados de 8 bits original.

## Parte 2

Através da implementação realizada na parte 1 exploraremos sua execução com dois modos de operação de cifra de blocos.

Os modos de operação permitem que o algoritmo criptográfico seja melhorado ou destinado a uma aplicação específica.

Ambos modos de operação a serem desenvolvidos neste relatório serão testados igualmente, portanto iniciaremos definindo as variáveis de teste:

In [12]:
# key is the same as before
message = '11010111011011001011101011110000'
iv = '01010101'

### CBC (Cipher Block Chaining):

Este modo de operação é muito semelhantes com uma diferença chave: com a saída do bloco anterior é feito um XOR com o plain-text do bloco seguinte. Logo, para o primeiro bloco na encriptação, faz-se necessário ter um vetor de inicialização o qual será usado para realizar o primeiro XOR.

Dessa forma, mesmo que os blocos sejam iguais, eles não irão produzir textos cifrados iguais o que dificulta reconhecimento de padrões.

Além disso, este modo de operação também exige realizar "padding" caso a quantidade de caracteres não seja divisível por 8 bits. Assim, a implementação é:

In [13]:
def encrypt_sdes_cbc(plain_text, key, iv):
    print(f'The plaintext being encrypted using CBC is: {plain_text}')

    # Padding with zeros
    if len(plain_text) % 8 != 0:
        plain_text += '0' * (8 - (len(plain_text) % 8))
    print(f'The plaintext after padding when needed is: {plain_text}')

    # Create blocks
    blocks = [plain_text[i: i + 8] for i in range(0, len(plain_text), 8)]

    # Encrypted blocks
    enc_blocks = []
    prev_cipher = iv
    for block in blocks:
        xor_result = ''.join([str(int(current) ^ int(prev)) for current, prev in zip(block, prev_cipher)])
        enc_blocks.append(enc(xor_result, key))
        prev_cipher = enc_blocks[-1]

    # Result
    cbc_cipher = ''.join(enc_blocks)
    print(f'The cipher result using CBC is: {cbc_cipher}')
    return cbc_cipher

encrypt_sdes_cbc(message, key, iv)

The plaintext being encrypted using CBC is: 11010111011011001011101011110000
The plaintext after padding when needed is: 11010111011011001011101011110000
The message being encoded is: 10000010
10-bit key: 1010000010
After P10: 1000001100
After LS-1: 0000111000
K1 = 10100100
After LS-2: 0010000011
K2 = 01000011
After IP: 00010001
The input is a 4-bit number: 0001
Matrix after E/P:
1000
0010
Matrix after XOR:
[0, 0, 1, 0]
[0, 1, 1, 0]
After S-boxes: 0011
After P4: 0110
Result after XOR of the F function and Left side: 0111
After fk1: 01110001
After SW: 00010111
The input is a 4-bit number: 0111
Matrix after E/P:
1011
1110
Matrix after XOR:
[1, 1, 1, 1]
[1, 1, 0, 1]
After S-boxes: 1000
After P4: 0001
Result after XOR of the F function and Left side: 0000
After fk2: 00000111
After the Inverse of IP: 00001011
The 8-bit ciphertext is: 00001011
The message being encoded is: 01100111
10-bit key: 1010000010
After P10: 1000001100
After LS-1: 0000111000
K1 = 10100100
After LS-2: 0010000011
K2 = 0

'00001011101010011001101101101010'

### ECB (Electronic Codebook):

Este modo de operação funciona dividindo o texto em blocos de tamanho definido pelo algoritmo de encriptação a ser utilizado. Após a divisão, cada bloco é cifrado de forma independente, mesmo que às vezes feito de forma paralela usando a mesma chave e o mesmo algoritmo.

Nesse sentido, o resultado deste modo de operação é a concatenação dos blocos cifrados.

Entretanto, apresenta um problema de segurança, pois blocos idênticos de textos simples como é o caso do teste do código abaixo geram blocos idênticos de texto cifrado. Com essas igualdades, é possível expor padrões o que compromete a segurança da encriptação.

Dessa forma, a função deve realizar "padding" caso a quantidade de caracteres não seja divisível por 8 bits e encriptar cada bloco e em seguida juntá-los, como pode ser visualizado abaixo:

In [14]:
def encrypt_sdes_ecb(plain_text, key):
    print(f'The plaintext being encrypted using ECB is: {plain_text}')

    # Padding with zeros
    if len(plain_text) % 8 != 0:
        plain_text += '0' * (8 - (len(plain_text) % 8))
    print(f'The plaintext after padding when needed is: {plain_text}')

    # Create blocks
    blocks = [plain_text[i: i + 8] for i in range(0, len(plain_text), 8)]

    # Encrypted blocks
    enc_blocks = [enc(block, key) for block in blocks]

    # Result
    ecb_cipher = ''.join(enc_blocks)
    print(f'The cipher result using ECB is: {ecb_cipher}')
    return ecb_cipher

encrypt_sdes_ecb(message, key)

The plaintext being encrypted using ECB is: 11010111011011001011101011110000
The plaintext after padding when needed is: 11010111011011001011101011110000
The message being encoded is: 11010111
10-bit key: 1010000010
After P10: 1000001100
After LS-1: 0000111000
K1 = 10100100
After LS-2: 0010000011
K2 = 01000011
After IP: 11011101
The input is a 4-bit number: 1101
Matrix after E/P:
1110
1011
Matrix after XOR:
[0, 1, 0, 0]
[1, 1, 1, 1]
After S-boxes: 1111
After P4: 1111
Result after XOR of the F function and Left side: 0010
After fk1: 00101101
After SW: 11010010
The input is a 4-bit number: 0010
Matrix after E/P:
0001
0100
Matrix after XOR:
[0, 1, 0, 1]
[0, 1, 1, 1]
After S-boxes: 0111
After P4: 1110
Result after XOR of the F function and Left side: 0011
After fk2: 00110010
After the Inverse of IP: 10101000
The 8-bit ciphertext is: 10101000
The message being encoded is: 01101100
10-bit key: 1010000010
After P10: 1000001100
After LS-1: 0000111000
K1 = 10100100
After LS-2: 0010000011
K2 = 0

'10101000000011010010111001101101'